In [1]:
cv_results = []
for i in range(5):
    cv_results.append({'train': {'logloss': ['0.48253', '0.35953']},
 'eval': {'logloss': ['0.480385', '0.357756']}})

In [4]:
    for result in cv_results:
        result_print = "".join("| {0}: {1} |".format(k, v) for k, v in result.items())
        print(result_print)

| train: {'logloss': ['0.48253', '0.35953']} || eval: {'logloss': ['0.480385', '0.357756']} |
| train: {'logloss': ['0.48253', '0.35953']} || eval: {'logloss': ['0.480385', '0.357756']} |
| train: {'logloss': ['0.48253', '0.35953']} || eval: {'logloss': ['0.480385', '0.357756']} |
| train: {'logloss': ['0.48253', '0.35953']} || eval: {'logloss': ['0.480385', '0.357756']} |
| train: {'logloss': ['0.48253', '0.35953']} || eval: {'logloss': ['0.480385', '0.357756']} |


In [24]:
for i, metrics in enumerate(cv_results):
    loss_eval = list(metrics["train"].values())
    print (loss_eval)

[['0.48253', '0.35953']]
[['0.48253', '0.35953']]
[['0.48253', '0.35953']]
[['0.48253', '0.35953']]
[['0.48253', '0.35953']]


In [33]:
for i, metrics in enumerate(cv_results):
    loss_eval = [x[-1] for x in list(metrics["train"].values())][0]
    print (loss_eval)

0.35953
0.35953
0.35953
0.35953
0.35953


In [37]:
for i, metrics in enumerate(cv_results):
    loss_eval = float([x[-1] for x in list(metrics["eval"].values())][0])
    loss_train = float([x[-1] for x in list(metrics["train"].values())][0])
    print ("{0}|\tEval:\t{1:.3f}\t|\ttrain:\t{2:.3f}"\
           .format(i, loss_eval, loss_train)) 

0|	Eval:	0.358	|	train:	0.360
1|	Eval:	0.358	|	train:	0.360
2|	Eval:	0.358	|	train:	0.360
3|	Eval:	0.358	|	train:	0.360
4|	Eval:	0.358	|	train:	0.360
